In [75]:
import xlrd
import pandas as pd
import re
import numpy as np

class POSReport:
    def __init__(self, file_path, header_row_index, columns_to_keep):
        self.file_path = file_path
        self.header_row_index = header_row_index
        self.columns_to_keep = columns_to_keep
        self.output_table = []
        self.df = pd.read_excel(self.file_path, header= self.header_row_index, usecols=self.columns_to_keep, dtype='str')


class InventoryList(POSReport):
    def __init__(self, file_path, header_row_index=4, columns_to_keep="A:C"):
        super().__init__(file_path, header_row_index, columns_to_keep)
        self.filter_df()

    def filter_df(self):
        """
        Filter Steps:
        - Disregard contents in columns D onwards
        - Remove blank rows
        - Remove irrelevat rows (non-item rows) such as column, category, total rows
        - Remove Duplicate Rows
        """
        self.df.dropna(thresh=3, inplace=True)
        # Drop Header Rows and Total Rows
        self.df = self.df.drop(self.df[(self.df.Category == 'Category') & (self.df.Name == 'Name') & (self.df.UPC == 'UPC')].index)
        self.df = self.df.drop(self.df[(self.df.Category == 'None') & (self.df.Name == 'Open Item - Transaction Fee')].index)
        self.df = self.df.drop(self.df[(self.df.Category == 'None') & (self.df.Name == 'Transaction Fee')].index)
        # Drop duplicate rows
        self.df.drop_duplicates(keep='last', inplace=True)
        return
        

class PriceList(POSReport):
    def __init__(self, file_path, header_row_index=3, columns_to_keep='A:C'):
        super().__init__(file_path, header_row_index, columns_to_keep)
        self.filter_df()

    def filter_df(self):
        """
        Filter Steps:
        - Disregard contents in columns D through G
        - Remove blank rows
        - Remove irrelevat rows (non-item rows) such as column, category, total rows
        - Strip/Upper each item name & size
        - Try and extract the unit/packaging size
        - Remove Duplicate Rows
        """
        self.df.rename(columns={"Item Name": "Name"}, inplace=True)
        # Drop blank rows
        self.df.dropna(thresh=2, inplace=True, subset=['Name', 'Price'])
        # Drop Header Rows and Total Rows
        self.df = self.df.drop(self.df[(self.df.Name == 'Item Name') & (self.df.Size == 'Size') & (self.df.Price == 'Price')].index)
        self.df = self.df.drop(self.df[(self.df.Name == 'None') & (self.df.Price == 'Open Item - Transaction Fee')].index)
        self.df = self.df.drop(self.df[(self.df.Name == 'None') & (self.df.Price == 'Transaction Fee')].index)
        # Drop duplicate rows
        self.df.drop_duplicates(keep='last', inplace=True)
        return
           

class SalesList(POSReport):
    def __init__(self, file_path, header_row_index=7, columns_to_keep='A:B'):
        super().__init__(file_path, header_row_index, columns_to_keep)
        self.filter_df()

    def filter_df(self):
        """
        Extract only the items and their UPC on the sales list
        """
        self.df.rename(columns={"Item Name": "Name"}, inplace=True)
        self.df.dropna(thresh=2, inplace=True)
        # Drop Header Rows and Total Rows
        self.df = self.df.drop(self.df[(self.df.UPC == 'UPC') & (self.df.Name == 'Item Name') & (self.df.UPC == 'UPC')].index)
        # Drop duplicate rows
        self.df.drop_duplicates(keep='last', inplace=True)
        return

pd.set_option('display.max_rows', 500)

In [76]:
inventory = InventoryList('./pos_reports/inventory/inv_10_14_2021.xls')
price = PriceList('./pos_reports/price/price_10_14_2021.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [77]:
# Get a df of all items sold in the past 6 months
sales_dfs = []
# Read in and append SalesList objects to the list
for i in range(4,10):
    sales_dfs.append(SalesList(f'./pos_reports/sales/sales_{i}.xls').df)

all_sales_items = pd.concat(sales_dfs)
all_sales_items = all_sales_items.drop_duplicates(keep='last')
print(len(all_sales_items))

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
3278


In [78]:
print(len(inventory.df))
print(len(all_sales_items))

6430
3278


In [79]:
# Remove inactive items
active_inventory = pd.merge(all_sales_items, inventory.df, on=['Name','UPC'], how='right', indicator='Active')
active_inventory['Active'] = np.where(active_inventory.Active == 'both', True, False)
active_inventory = active_inventory[active_inventory['Active']==True]
active_inventory.drop("Active", axis=1, inplace=True)

In [80]:
# Create a column that marks items with duplicates (need price manually changed)
active_inventory['Duplicate'] = active_inventory['Name'].duplicated()

In [81]:
print(len(active_inventory))

3273


In [82]:
complete_list = pd.merge(active_inventory, price.df,  how='left', on='Name', validate='many_to_many')
complete_list.drop_duplicates(subset=['UPC', 'Name'], keep='first', inplace=True)
print(complete_list[complete_list.Duplicate==True])

               UPC                                   Name      Category  \
67     18200250002                              BUDWEISER          BEER   
71     18200110474                              BUDWEISER          BEER   
79    810960024561                         BVR PSNFRT GVA          BEER   
94     71990300654                            COORS LIGHT          BEER   
98     71990300692                            COORS LIGHT          BEER   
110    71990000141                     COORS SELTZER 24OZ          BEER   
124    33544000687                   CORONA SELTZER 12CAN          BEER   
147   602755006353                          FIRESTONE 805          BEER   
175   723830000094                          LAGUNITAS IPA          BEER   
221    34100575090                            MILLER LITE          BEER   
223    34100576363                            MILLER LITE          BEER   
267    33544950227                               PACIFICO          BEER   
270    80660953014       